```
# Copyright 2022 by the author(s) of CHI2023 submission "Short-Form
# Videos Degrade Our Capacity to Retain Intentions: Effect of Context
# Switching On Prospective Memory". All rights reserved.
#
# Use of this source code is governed by a GPLv3 license that
# can be found in the LICENSE file.
```

This file provides the code to fit DDM mdoel.

In [ ]:
from typing import List
import pandas as pd

from pyddm import Sample, Model, Fittable
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, LossRobustBIC
from pyddm.functions import fit_adjust_model


In [ ]:
def fit_ddm(df: pd.DataFrame, rt_column_name: str, correct_column_name: str) -> List[float]:
    samples = Sample.from_pandas_dataframe(df, rt_column_name=rt_column_name, correct_column_name=correct_column_name)
    m = Model(name='ddm',
        drift=DriftConstant(drift=Fittable(minval=0, maxval=50)),
        noise=NoiseConstant(noise=Fittable(minval=.5, maxval=4)),
        bound=BoundConstant(B=Fittable(minval=.1, maxval=2)),
        overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
        dx=.001, dt=.0001, T_dur=3,
    )
    fit_adjust_model(samples, m, fitting_method="differential_evolution", lossfunction=LossRobustBIC, verbose=False)
    # drift, noise, bound, nondecisiontime
    return [p.default() for p in m.get_model_parameters()]

In [ ]:

df_rt = pd.read_csv('data/rt.csv')

df_ddm = pd.DataFrame(columns=['folder_id', 'interrupt', 'task', 'measure', 'drift', 'noise', 'bound', 'nondectime'])
for folder_id in df_rt.folder_id.unique():
    for measure in ['pre', 'post']:
        for task in ['LD', 'PM']:
            df_data = df_rt[(df_rt.task == task) & (df_rt.folder_id == folder_id) & (df_rt.measure == measure) & (df_rt.success == True)]
            params = fit_ddm(df_data.dropna(), 'rt', 'correct')
            if str(folder_id)[0] == '1':
                interrupt = 'rest'
            elif str(folder_id)[0] == '2':
                interrupt = 'tiktok'
            elif str(folder_id)[0] == '3':
                interrupt = 'youtube'
            elif str(folder_id)[0] == '4':
                interrupt = 'twitter'

            df_ddm = df_ddm.append({
                'folder_id': folder_id,
                'interrupt': interrupt,
                'task': task,
                'measure': measure,
                'drift': params[0],
                'noise': params[1],
                'bound': params[2],
                'nondectime': params[3]}, ignore_index=True)
            df_ddm.to_csv('data/ddm.csv', index=False)

In [ ]:
df_ddm